In [151]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [152]:
from sklearn.datasets import load_iris
X, Y = load_iris()['data'], load_iris()['target']
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [153]:
from sklearn.base import ClassifierMixin

class SVM(ClassifierMixin):
    def __init__(self, lam=0.01, lr=0.1, epochs=200):
        super(SVM).__init__()
        self.lam = lam
        self.lr = lr
        self.epochs = epochs
        self.w = None
        self.b = None
        self.mapper = dict()

    def map_classes(self, Y):
        classes = list(set(Y))
        Y = (Y == classes[0]).astype(int)
        Y[Y==0] = -1
        self.mapper[1] = classes[0]
        self.mapper[-1] = classes[1]
        return Y
    
    def demap_classes(self, ans):
        return np.array([self.mapper[1] if i == 1 else self.mapper[-1] for i in ans])


    def fit(self, X,Y):
        Y = self.map_classes(Y)
        n,d = X.shape
        self.w = np.random.random(d)
        self.b = 0
        for _ in range(self.epochs):
            idx = np.random.permutation(n)
            for i in idx:
                margin = Y[i] * ( np.dot(X[i],self.w) + self.b )
                w_grad = 2*self.lam*self.w 
                b_grad = 0
                if margin < 1:
                    w_grad -= Y[i] * X[i]
                    b_grad -= Y[i]
                self.w -= self.lr * w_grad
                self.b -= self.lr * b_grad

        return self

    def predict(self, X):
        ans = []
        for i in range(len(X)):
            ans.append(float(np.sign((np.dot(X[i], self.w) + self.b))))
        return self.demap_classes(ans)


In [154]:
svm = SVM()
svm.fit(X,Y)
pred = svm.predict(X)
pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [155]:
from sklearn.metrics import f1_score, recall_score, accuracy_score, precision_score
# print(f1_score(y_true=Y, y_pred=pred))
# print(recall_score(y_true=Y, y_pred=pred))
# print(accuracy_score(y_true=Y, y_pred=pred))
# print(precision_score(y_true=Y, y_pred=pred))

In [156]:
svm.w, svm.b

(array([ 0.13376252,  1.22546416, -1.80775862, -0.88172913]),
 np.float64(2.400000000000001))

In [157]:
def print_hyperplane(svms):
    def get_hyperplane_value(x, w, b, offset):
        return -(w[0] * x + b + offset) / w[1]

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    plt.scatter(X[:, 0], X[:, 1], marker="o", c=Y)

    x0_1 = np.amin(X[:, 0])
    x0_2 = np.amax(X[:, 0])
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']
    for i, svm in enumerate(svms):
        
        x1_1 = get_hyperplane_value(x0_1, svm.w, svm.b, 0)
        x1_2 = get_hyperplane_value(x0_2, svm.w, svm.b, 0)
        ax.plot([x0_1, x0_2], [x1_1, x1_2], colors[i])

    x1_min = np.amin(X[:, 1])
    x1_max = np.amax(X[:, 1])
    ax.set_ylim([x1_min - 3, x1_max + 3])

    plt.show()

 Окей, получился svm, который пытается разделить объекты на два класса. Обобщаем для многоклассовой классификации

<h1>"Все против всех" или "Один против всех" ??</h1> 

В случае один против всех будет слишком большой дисбаланс классов, что приведет к нелучшим результатам. А метод Все против всех у нас количество классификаторов растет квадратично. Поэтому нужно решить, что нам подходит больше. Ествественно, если у нас классов меньше, например, 5, лучше использовать all-vs-all. В айрис датасете всего 3 класса, поэтому почему бы не пожертвовать 3 секундами вычисления для более точной классификации


Напишем класс, который будет принимать полный датасет и разделять метки классов на пары, после чего обрабатывать эти пары нашей svm

In [158]:
from itertools import combinations
class all_vs_all(ClassifierMixin):
    def __init__(self, lam=0.01, lr=0.1, epochs=200):
        super(all_vs_all).__init__()
        self.svms = []
        self.lam = lam
        self.lr = lr
        self.epochs = epochs

    def print_score(self, svm, x, y, cl1,cl2):
        pred = svm.predict(x)
        print(f"SVM classes {cl1} and {cl2}")
        print('f1', f1_score(y_true=y, y_pred=pred, pos_label = cl1))
        print('recall', recall_score(y_true=y, y_pred=pred, pos_label = cl1))
        print('accuracy', accuracy_score(y_true=y, y_pred=pred))
        print('precision', precision_score(y_true=y, y_pred=pred, pos_label = cl1), '\n')

    def print_hyperplanes(self,):
        print_hyperplane(self.svms)
        
    def fit(self, X:np.ndarray, Y:np.ndarray, pr=False):
        combs = list(combinations(set([int(i) for i in list(Y)]), 2))
        self.svms = []
        for cl1, cl2 in combs:
            svm = SVM(self.lam, self.lr, self.epochs)
            y = np.concat([Y[Y == cl1],Y[Y==cl2]])
            x = np.concat([X[Y == cl1],X[Y==cl2]])
            svm.fit(X=x, Y=y)
            self.svms.append(svm)
            if pr: 
                self.print_score(svm,x,y,cl1,cl2)


In [162]:
clf = all_vs_all()
clf.fit(X,Y, pr=True)


SVM classes 0 and 1
f1 1.0
recall 1.0
accuracy 1.0
precision 1.0 

SVM classes 0 and 2
f1 1.0
recall 1.0
accuracy 1.0
precision 1.0 

SVM classes 1 and 2
f1 0.8849557522123894
recall 1.0
accuracy 0.87
precision 0.7936507936507936 

